In [24]:
import pandas as pd
import regex as re

pd.set_option('display.max_colwidth', None)
file = '/content/drive/MyDrive/DataSets/who_2020.xlsx'
df = pd.read_excel(file, sheet_name='Annex 2-2', header=1)
delete = list(range(5,20))
df=df.drop(df.columns[delete], axis=1)
df.drop(index=[0,1], inplace=True)
df=df.rename(columns={'Unnamed: 0':'Country'})
df.drop(index=[2], inplace=True)
df.set_index('Country')
df=df[~df['Country'].str.contains('Region', flags=re.I, regex=True, na=False)]
df.drop(index=[197,205], inplace=True)

In [25]:
import requests
from bs4 import BeautifulSoup

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_European_Union_member_states_by_population').text
soup = BeautifulSoup(website_url,'lxml')

my_table = soup.find('table',{'class':'sortable wikitable'})
links = my_table.findAll('a')

countries = []
for link in links:
  countries.append(link.get('title'))

dfc = pd.DataFrame()
dfc['countries'] = countries
u = dfc['countries'].unique()

dfd = pd.DataFrame(u)
dfd.rename(columns={'0':'Country'}, inplace=True)
dfd.drop(index=0, inplace=True)
dfd.columns=['Country']

In [ ]:
df['EU?']=df['Country'].isin(dfd['Country'])
df_eu = df[df['EU?']==True]
df_eu

In [33]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Choropleth(
    locations=df_eu['Country'], # Spatial coordinates
    z = df_eu['3'], # Data to be color-coded
    locationmode = 'country names', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "Sightings",
))

fig.update_layout(
    scope='europe', # limite map scope to USA
)

fig.show()

KeyError: ignored